In [1]:
import pandas as pd

# 파일 불러오기
df = pd.read_csv('기상데이터.csv', header=None, encoding='cp949')

# 컬럼명 지정
columns = [
    '지점', '지점명', '일시', '기온(°C)', '강수량(mm)', '풍속(m/s)', 
    '풍향(16방위)', '습도(%)', '증기압(hPa)', '이슬점온도(°C)', 
    '현지기압(hPa)', '일조(hr)', '일사(MJ/m2)', '적설(cm)', 
    '전운량(10분위)', '중하층운량(10분위)', '최저운고(100m)'
]
df.columns = columns

# 1. 강수량, 적설만 NaN → 0 처리
df['강수량(mm)'] = df['강수량(mm)'].fillna(0)
df['적설(cm)'] = df['적설(cm)'].fillna(0)

# 2. 나머지 주요 컬럼 중 NaN 있는 행 제거
cols_with_critical_nan = [
    '기온(°C)', '풍속(m/s)', '풍향(16방위)', '습도(%)',
    '증기압(hPa)', '이슬점온도(°C)', '현지기압(hPa)'
]
df_cleaned = df.dropna(subset=cols_with_critical_nan)

# 결과 확인
print(f"✅ 결측치 처리 완료")
print(f"   - 강수량/적설 NaN → 0")
print(f"   - 결측치 있는 행 제거 후: {len(df_cleaned):,}건 남음")

# 저장
df_cleaned.to_csv('기상데이터_처리완료.csv', index=False, encoding='utf-8-sig')


✅ 결측치 처리 완료
   - 강수량/적설 NaN → 0
   - 결측치 있는 행 제거 후: 441,790건 남음


In [2]:
# 풍력발전 데이터 읽기
wind_df = pd.read_csv('unified_wind_power_data.csv')

# 기상데이터_처리완료.csv 파일 읽기
weather_df = pd.read_csv('기상데이터_처리완료.csv')

# 발전구분과 지점명 매핑 정의
station_mapping = {
    '어음풍력': '제주',
    '군위 화산풍력': '의성',
    '한경풍력': '고산',
    '성산풍력': '서귀포',
    '화순풍력': '광주',
    '상명풍력': '고산'
}

print(f"\n=== 발전구분별 매핑 ===")
for plant, station in station_mapping.items():
    print(f"   {plant} → {station}")

# 풍력발전 데이터에 지점명 컬럼 추가
wind_df['지점명'] = wind_df['발전구분'].map(station_mapping)

# 매핑되지 않은 발전구분 확인 및 제거
unmapped = wind_df[wind_df['지점명'].isna()]['발전구분'].unique()
if len(unmapped) > 0:
    print(f"\n⚠️ 매핑되지 않은 발전구분 제거: {unmapped}")
    wind_df = wind_df[wind_df['지점명'].notna()]  # 매핑되지 않은 행 제거
    print(f"   제거 후 풍력발전 데이터: {len(wind_df):,}건")

# 기상데이터의 지점명 확인
print(f"\n=== 기상데이터 지점명 ===")
print(f"   지점명 종류: {weather_df['지점명'].unique()}")

# 매핑되는 지점명만 필터링 (기상데이터)
mapped_stations = list(station_mapping.values())
weather_df = weather_df[weather_df['지점명'].isin(mapped_stations)]
print(f"   매핑되는 지점만 필터링 후: {len(weather_df):,}건")

# 일시 컬럼을 datetime으로 변환 (기상데이터)
weather_df['일시'] = pd.to_datetime(weather_df['일시'])
weather_df['날짜'] = weather_df['일시'].dt.date
weather_df['시간'] = weather_df['일시'].dt.hour + 1  # 1시~24시로 맞추기

# 풍력발전 데이터의 일자 컬럼을 datetime으로 변환
wind_df['일자'] = pd.to_datetime(wind_df['일자'])
wind_df['날짜'] = wind_df['일자'].dt.date

print(f"\n=== 시간 정보 변환 완료 ===")
print(f"   기상데이터 날짜 범위: {weather_df['날짜'].min()} ~ {weather_df['날짜'].max()}")
print(f"   풍력발전 데이터 날짜 범위: {wind_df['날짜'].min()} ~ {wind_df['날짜'].max()}")

# 두 데이터를 지점명, 날짜, 시간으로 병합
merged_df = pd.merge(
    wind_df,
    weather_df,
    on=['지점명', '날짜', '시간'],
    how='left'
)

print(f"\n=== 데이터 병합 완료 ===")
print(f"   풍력발전 데이터: {len(wind_df):,}건")
print(f"   기상데이터: {len(weather_df):,}건")
print(f"   병합된 데이터: {len(merged_df):,}건")

# 병합 결과 확인
print(f"\n=== 발전구분별 병합 결과 ===")
for plant in merged_df['발전구분'].unique():
    plant_data = merged_df[merged_df['발전구분'] == plant]
    station = plant_data['지점명'].iloc[0] if len(plant_data) > 0 else 'Unknown'
    weather_missing = plant_data['기온(°C)'].isna().sum()
    print(f"   {plant} ({station}): {len(plant_data):,}건, 기상데이터 결측: {weather_missing}건")


# 최종 데이터 저장
merged_df.to_csv('기상과풍력.csv', index=False, encoding='utf-8-sig')

print(f"\n✅ 풍력발전-기상데이터 통합 완료!")
print(f"   - 저장 파일: 기상과풍력.csv")
print(f"   - 총 데이터 건수: {len(merged_df):,}건")

# 샘플 데이터 확인
print(f"\n=== 통합 데이터 샘플 ===")
sample_cols = ['발전구분', '지점명', '일자', '시간', '발전량(kWh)', '설비용량(MW)', 
               '기온(°C)', '풍속(m/s)', '습도(%)']
print(merged_df[sample_cols].head())

print(f"\n=== 최종 컬럼 목록 ===")
print(f"   총 {len(merged_df.columns)}개 컬럼:")
for i, col in enumerate(merged_df.columns, 1):
    print(f"   {i:2d}. {col}")


=== 발전구분별 매핑 ===
   어음풍력 → 제주
   군위 화산풍력 → 의성
   한경풍력 → 고산
   성산풍력 → 서귀포
   화순풍력 → 광주
   상명풍력 → 고산

=== 기상데이터 지점명 ===
   지점명 종류: ['제주' '의성' '서귀포' '고산' '광주']
   매핑되는 지점만 필터링 후: 441,790건

=== 시간 정보 변환 완료 ===
   기상데이터 날짜 범위: 2013-01-01 ~ 2025-06-01
   풍력발전 데이터 날짜 범위: 2013-01-01 ~ 2025-05-31

=== 데이터 병합 완료 ===
   풍력발전 데이터: 496,560건
   기상데이터: 441,790건
   병합된 데이터: 496,560건

=== 발전구분별 병합 결과 ===
   군위 화산풍력 (의성): 29,928건, 기상데이터 결측: 10건
   어음풍력 (제주): 13,128건, 기상데이터 결측: 8건
   상명풍력 (고산): 18,288건, 기상데이터 결측: 356건
   한경풍력 (고산): 213,240건, 기상데이터 결측: 2740건
   성산풍력 (서귀포): 213,216건, 기상데이터 결측: 494건
   화순풍력 (광주): 8,760건, 기상데이터 결측: 32건

✅ 풍력발전-기상데이터 통합 완료!
   - 저장 파일: 기상과풍력.csv
   - 총 데이터 건수: 496,560건

=== 통합 데이터 샘플 ===
      발전구분 지점명         일자  시간  발전량(kWh)  설비용량(MW)  기온(°C)  풍속(m/s)  습도(%)
0  군위 화산풍력  의성 2025-05-31   1       0.0      1.65    12.4      1.2   87.0
1  군위 화산풍력  의성 2025-05-31   2       0.0      1.65    11.7      0.3   93.0
2  군위 화산풍력  의성 2025-05-31   3       0.0      1.65    10.6      0.8   9

화순풍력 기상데이터 매핑이 안됨